In [17]:
# import arcpy
import pandas as pd # pandas upgrade required to use .query, PROPY env does not like that version 
import geopandas as gpd
import os
import csv

import sys
sys.path.insert(0,'M:/code/USFS')
from helpers import set_years, MakeIterDir

In [34]:
os.getcwd()

'M:\\projects\\data_processing'

ModuleNotFoundError: No module named 'ee'

In [12]:
lcdates_csv = r"M:\projects\tc_fact_sheet\data\county_list_legendloc.csv"
codf = pd.read_csv(lcdates_csv)

omit = ['wash_11001']
maintain = ['bedf_51515', 'lanc_51103']
codf = codf[~codf['cofips'].isin(omit) & codf['cofips'].isin(maintain)]
codf = codf[codf.Size.isin(['s','xs'])]
print(len(codf), " counties in codf")



1  counties in codf


In [ ]:


#WITH SAFETIES

pngDIR = "M:/projects/tc_fact_sheet/map_exports/auto"
IterationDir = MakeIterDir(pngDIR, "test")

fail_list = []
counter = 0

for index, row in codf.iterrows():
    cf = row['cofips']
    T1 = row['T1']
    T2 = row['T2']
    QA = row['QA']
    size = row['Size']
    
    if size == 
    
    FIPS = cf.split('_')[1]
    counter += 1
#     if counter > 5: # for testing
#         break
    print(f"{cf} {counter}/{len(codf)}", end='\r')

    FIPS = cf.split('_')[1]
    tempDir = f"M:/projects/tc_fact_sheet/data/temp"
    coDir = f"M:/projects/tc_fact_sheet/data/prod/{cf}_{hex_ac}"
    CoBoundsPath = r"J:/GIS/CBP_Obj_1/data/county_sets/tl_2017_bay_county.shp"

    if not os.path.isdir(coDir):
        os.mkdir(coDir)


    CoSel = os.path.join(tempDir,f"{cf}_bounds.shp")

    where_clause = f"""GEOID = '{FIPS}'"""
    print(where_clause)

    BayHexPath = f"M:/projects/data_processing/data_processing.gdb/baywide_{hex_ac}"
    CoHexPath = os.path.join(coDir,f"{cf}_hex.shp")
    calcCSVPath = os.path.join(coDir,f"{cf}_ta_calc.csv")
    ta_dbf_path = os.path.join(coDir,f"{cf}_hex_ta.dbf")
    ta_xlsx_path = ta_dbf_path.replace(".dbf",".xlsx")
    lulc_chgPath = f"B:/landuse/version2/{cf}/output/{cf}_landusechange_{T1}{T2}.tif"

    t2luPath = f"B:/landuse/version2/{cf}/output/{cf}_lu_{T2}.tif"
    t2_dbf_path = os.path.join(coDir,f"{cf}_hex_t2_ta.dbf")
    t2_xlsx_path = t2_dbf_path.replace(".dbf",".xlsx")
    t2_csv_path = t2_xlsx_path.replace('xlsx','csv')

    tajoined3Path = os.path.join(coDir,f"{cf}_hex_tajoined3.shp")

    if not arcpy.Exists(CoSel):
        sel_temp = arcpy.management.SelectLayerByAttribute(CoBoundsPath, "NEW_SELECTION", where_clause)
        arcpy.FeatureClassToFeatureClass_conversion(sel_temp, tempDir, os.path.basename(CoSel))
    else:
        print(f"{CoSel} exists.")

    if not arcpy.Exists(CoHexPath):
        tempHex = arcpy.SelectLayerByLocation_management(BayHexPath, 'INTERSECT', CoSel)
        arcpy.FeatureClassToFeatureClass_conversion(tempHex, coDir, os.path.basename(CoHexPath))
        del tempHex

    
    if not os.path.exists(ta_dbf_path):
        print(f"TA - {ta_dbf_path}")
        arcpy.sa.TabulateArea(CoHexPath, 
                            "GRID_ID", 
                            lulc_chgPath, 
                            "Value", 
                            ta_dbf_path, 
                            lulc_chgPath, 
                            "CLASSES_AS_FIELDS")


    if not os.path.exists(t2_dbf_path):
        print(f"TA - {ta_dbf_path}")
        arcpy.sa.TabulateArea(CoHexPath, 
                            "GRID_ID", 
                            t2luPath, 
                            "Value", 
                            t2_dbf_path, 
                            t2luPath, 
                            "CLASSES_AS_FIELDS")

    # Convert to excel format to visually inspect easier and read into pandas
    if not os.path.exists(ta_xlsx_path):
        arcpy.conversion.TableToExcel(ta_dbf_path, ta_xlsx_path, "NAME", "CODE")
    if not os.path.exists(t2_xlsx_path):   
        arcpy.conversion.TableToExcel(t2_dbf_path, t2_xlsx_path, "NAME", "CODE")

In [16]:
elist = [] #clear elist
counter = 0

tc = [24,25,26,27,41,42]
dev = [21,22,23,28,29,33,34,35,36,37,38,51,52,53]
wat = [11]

for index, row in codf.iterrows():
    cf = row['cofips']
    print(cf)
    T1 = row['T1']
    T2 = row['T2']
    QA = row['QA']
    size = row['Size']
    FIPS = cf.split('_')[1]
    T1,T2 = set_years(cf)

    counter += 1    

    # BUILD CLASS so you don't have to define these for each loop
    coDir = f"M:/projects/tc_fact_sheet/data/prod/{cf}_{hex_ac}"
    CoHexPath = os.path.join(coDir,f"{cf}_hex.shp")
    BayHexPath = f"M:/projects/data_processing/data_processing.gdb/baywide_{hex_ac}"
    calcCSVPath = os.path.join(coDir,f"{cf}_ta_calc.csv")
    ta_dbf_path = os.path.join(coDir,f"{cf}_hex_ta.dbf")
    ta_xlsx_path = ta_dbf_path.replace(".dbf",".xlsx")
    lulc_chgPath = f"B:/landuse/version2/{cf}/output/{cf}_landusechange_{T1}{T2}.tif"

    t2luPath = f"B:/landuse/version2/{cf}/output/{cf}_lu_{T2}.tif"
    t2_dbf_path = os.path.join(coDir,f"{cf}_hex_t2_ta.dbf")
    t2_xlsx_path = t2_dbf_path.replace(".dbf",".xlsx")
    t2_csv_path = t2_xlsx_path.replace('xlsx','csv')
    tajoined3Path = os.path.join(coDir,f"{cf}_hex_tajoined3.shp")

    # #read excel files to be edited
    # tadf = pd.read_excel(ta_xlsx_path)
    # t2df = pd.read_excel(t2_xlsx_path)

    # #modify ta results for LULC Change
    # tadf['tc_chg'] = 0
    # tavals = [col for col in tadf.columns if col[0:5] == "VALUE"]
    # loss_vals = []
    # gain_vals = []
    # # for tv in list(tadf.VALUE.unique()):
    # for tv in tavals:
    # ## If it contains the VALUE prefix use this section
    #     T1val = int(tv[6:8])
    #     T2val = int(tv[8:10])
    # #     T1val = int(str(tv)[0:2])
    # #     T2val = int(str(tv)[2:4])
    #     if T1val in tc and T2val in dev:
    #         loss_vals.append(tv)
    #     elif T1val in dev and T2val in tc:
    #         gain_vals.append(tv)
    #     else:
    #         pass

    # tadf['tc_loss']=tadf[loss_vals].sum(axis=1)
    # tadf['tc_gain']=tadf[gain_vals].sum(axis=1)
    # tadf['tc_chg']=tadf['tc_gain']-tadf['tc_loss']

    # tadf_clean = tadf[['GRID_ID','tc_chg','tc_gain','tc_loss']]
    # tadf_clean.to_csv(calcCSVPath,index=True)

    # # modify ta results for T2 LULC

    # if not os.path.exists(t2_csv_path):  
    #     t2df = pd.read_excel(t2_xlsx_path)
    #     t2vals = [col for col in t2df.columns if col[0:5] == "VALUE"]
        
    #     possible_wat_vals = ["VALUE_11","VALUE_12","VALUE_13","VALUE_14","VALUE_15"]
    #     wat_vals = [i for i in t2df.columns if i in possible_wat_vals]

    #     t2df['wat_lu'] = t2df[wat_vals].sum(axis=1)
    #     t2df['all_lu']= t2df[t2vals].sum(axis=1)
    #     t2df.to_csv(t2_csv_path)
    # else:
    #     t2df = pd.read_excel(t2_xlsx_path)


    # if not os.path.exists(tajoined3Path):  
    t2_csv_df = pd.read_csv(t2_csv_path)
    ta_csv_df = pd.read_csv(calcCSVPath)
    hex_df = gpd.read_file(CoHexPath)

    hex_df = hex_df.merge(t2_csv_df,how="left",left_on="GRID_ID",right_on="GRID_ID")
    hex_df = hex_df.merge(ta_csv_df,how="left",left_on="GRID_ID",right_on="GRID_ID")

    hex_df.to_file(tajoined3Path)


print('complete')
print(elist)



alex_51510


c:\Users\jczawlytko\.conda\envs\open-gis\lib\site-packages\ipykernel_launcher.py:89: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


complete
[]
arli_51013
complete
[]
balt_24510
complete
[]
buen_51530
complete
[]
char_51540
complete
[]
colo_51570
complete
[]
covi_51580
complete
[]
fair_51600
complete
[]
fall_51610
complete
[]
fred_51630
complete
[]
hamp_51650
complete
[]
harr_51660
complete
[]
hope_51670
complete
[]
lexi_51678
complete
[]
lync_51680
complete
[]
mana_51683
complete
[]
mana_51685
complete
[]
mont_42093
complete
[]
newp_51700
complete
[]
norf_51710
complete
[]
pete_51730
complete
[]
poqu_51735
complete
[]
port_51740
complete
[]
rich_51760
complete
[]
roan_51770
complete
[]
sale_51775
complete
[]
stau_51790
complete
[]
wayn_51820
complete
[]
will_51830
complete
[]


In [14]:
hex_df

,Shape_Leng,Shape_Area,GRID_ID,geometry,Unnamed: 0_x,OID,VALUE_13,VALUE_14,VALUE_15,VALUE_21,...,VALUE_74,VALUE_83,VALUE_84,VALUE_85,wat_lu,all_lu,Unnamed: 0_y,tc_chg,tc_gain,tc_loss
0,2368.014715,404685.999091,UF-897,"POLYGON ((1517507.388 1932039.878, 1517310.054...",0,0,0,0,0,0,...,0,0,573,0,0,15323,NaN,NaN,NaN,NaN
1,2368.014715,404685.999091,UF-896,"POLYGON ((1517507.388 1932723.465, 1517310.054...",1,1,0,0,0,6882,...,0,0,22671,0,0,214977,0.0,114.0,279.0,165.0
2,2368.014642,404685.974073,UD-895,"POLYGON ((1516323.381 1933407.052, 1516126.046...",2,2,0,16,0,31160,...,0,0,149593,0,16,334349,1.0,907.0,930.0,23.0
3,2368.014542,404685.939892,UE-895,"POLYGON ((1516915.385 1933065.258, 1516718.050...",3,3,0,0,0,5220,...,0,0,0,0,0,187669,2.0,286.0,286.0,0.0
4,2368.014542,404685.939892,UF-895,"POLYGON ((1517507.388 1933407.052, 1517310.054...",4,4,0,0,0,18557,...,0,0,0,0,0,404849,3.0,1936.0,2214.0,278.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2368.014715,404685.999092,UF-885,"POLYGON ((1517507.388 1940242.921, 1517310.054...",74,74,0,0,0,8782,...,0,0,72,5,0,101799,72.0,-195.0,105.0,300.0
75,2368.014815,404686.033272,UG-885,"POLYGON ((1518099.392 1939901.128, 1517902.057...",75,75,0,0,0,62955,...,0,0,0,0,0,336695,73.0,69.0,1000.0,931.0
76,2368.014815,404686.033272,UH-885,"POLYGON ((1518691.396 1940242.921, 1518494.061...",76,76,0,0,0,18,...,0,0,0,0,0,256,NaN,NaN,NaN,NaN
77,2368.014815,404686.033271,UC-884,"POLYGON ((1515731.377 1940584.715, 1515534.043...",77,77,0,0,0,0,...,0,0,2901,0,0,3699,NaN,NaN,NaN,NaN
